Compute a flag based on seed clustereing

In [1]:
import os
from pathlib import Path
import numpy as np
import requests
import pandas as pd

In [27]:
# Set path to data folder
current_dir = Path(os.getcwd())
DATA_DIR = os.path.join(current_dir.parent.parent, 'data-regen-rangers')
DATA_DIR_GITCOIN = os.path.join(current_dir.parent.parent, 'data-gitcoin')
GRANT_ROUND_ID = '0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc'
ODC_DATA_DIR = os.path.join(current_dir.parent.parent.parent, 'data')

PATH_TO_VOTES = os.path.join(DATA_DIR_GITCOIN, 'citizen-votes.csv')
# PATH_TO_GRANTS = os.path.join(DATA_DIR, "all-allo-rounds.csv")
# PATH_TO_PROJECTS = os.path.join(DATA_DIR, "projects_QmQurt.csv")

In [8]:
df_matching_address = pd.read_csv(f'../output_gitcoin/full_features/voters_features_{GRANT_ROUND_ID}.csv')
df_votes = pd.read_csv(f'../output_gitcoin/full_features/votes_features_citizen_{GRANT_ROUND_ID}.csv')
df_seed = pd.read_csv(os.path.join(DATA_DIR, 'seed_wallet_citizen.csv'))

In [6]:
df_matching_address.columns

Index(['address', 'seed_same_naive', 'seed_same', 'seed_suspicious',
       'less_5_tx', 'less_10_tx', 'interacted_other_ctbt', 'lcs',
       'cluster_size_lcs', 'mean_score_lcs', 'max_score_lcs', 'has_lcs',
       'count_interaction_with_pool', 'count_interaction_with_toxic',
       'interact_less_5tx', 'has_interaction_toxic', 'has_no_pool_interaction',
       'count_interaction_with_airdrop_m', 'is_airdrop_master',
       'count_interaction_with_tornado', 'count_interaction_with_disperse',
       'has_interaction_airdrop_m', 'has_interaction_tornado',
       'has_interaction_disperse', 'flagged'],
      dtype='object')

In [7]:
df_votes.columns

Index(['block_timestamp', 'tx_hash', 'voter', 'project', 'amount_usd',
       '__row_index', 'seed_same_naive', 'seed_same', 'seed_suspicious',
       'less_5_tx', 'less_10_tx', 'interacted_other_ctbt', 'lcs',
       'cluster_size_lcs', 'mean_score_lcs', 'max_score_lcs', 'has_lcs',
       'count_interaction_with_pool', 'count_interaction_with_toxic',
       'interact_less_5tx', 'has_interaction_toxic', 'has_no_pool_interaction',
       'count_interaction_with_airdrop_m', 'is_airdrop_master',
       'count_interaction_with_tornado', 'count_interaction_with_disperse',
       'has_interaction_airdrop_m', 'has_interaction_tornado',
       'has_interaction_disperse', 'flagged', 'stakeridoo_detected',
       'doge_detected', 'odc_detected'],
      dtype='object')

In [9]:
df_seed.head(2)

,EOA,from_address,to_address
0,0x000000006f457c0f8f560333d9c2877287d92a92,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x000000006f457c0f8f560333d9c2877287d92a92
1,0x000128fa45d79dc9af8016da242781f12c363fd5,0xe4edb277e41dc89ab076a1f049f4a3efa700bce8,0x000128fa45d79dc9af8016da242781f12c363fd5


EOA is the address of the citizen voter. From address is the address of the seeder.

In [63]:
print(df_matching_address['address'].shape)
print(df_matching_address['address'].unique().shape)
print(df_seed['EOA'].shape)
print(df_seed['EOA'].unique().shape)

(17023,)
(17023,)
(13820,)
(13820,)


In [19]:
np.setdiff1d(df_matching_address['address'], df_seed['EOA'])

array(['0x00773dcbbbb0ffb5d669e990c35d8339f080d1ee',
       '0x00781d50a170f09db92965094a1fa13bb119600b',
       '0x008076097a7a46993ecb1d1c21a7d3dd655afae3', ...,
       '0xffc0856eef7b73d0613bdd0f20fafa8b82f8dde7',
       '0xffd5a033e8ed8db8f63a1d19e189ffb82a06a779',
       '0xfff0016550753ce0b4b70704ae88aaf154bec4bf'], dtype=object)

**These addresses don't have any incomming transactions this explains what they don't have a seed wallet. We would need to explore normal transactions to find the seed wallet.**



We will consider that these wallet can't be flagged with the new seed clustering algorithm.

In [15]:
np.setdiff1d(df_matching_address['address'], df_votes.voter.unique()).shape

(0,)

In [25]:
df_count_seed = df_seed.groupby('from_address').count().sort_values(by='EOA', ascending=False).drop(columns='to_address')
df_count_seed.head(2)

,EOA
from_address,
0x80c67432656d59144ceff962e8faf8926599bcf8,3148
0xacd03d601e5bb1b275bb94076ff46ed9d753435a,2980


In [29]:
# load labels for EOA to know if it is a bridge or an exchange or something else
df_labels = pd.read_csv(os.path.join(ODC_DATA_DIR, 'address_labels_citizen.csv'), usecols=['address', 'tag', 'sub_type'])
df_labels_test = pd.read_csv(os.path.join(ODC_DATA_DIR, 'address_labels_citizen_test.csv'), usecols=['address', 'tag', 'sub_type'])
df_labels = pd.concat([df_labels, df_labels_test])
print(df_labels.shape)
df_labels.drop_duplicates(inplace=True)
print(df_labels.shape)

(105, 3)
(104, 3)


In [33]:
df_labels['cex_or_bridge'] = df_labels['tag'].apply(lambda x: True if x in ['cex_or_bridge', 'bridge', 'cex'] else False)

In [41]:
array_address_cex_or_bridge = df_labels[df_labels['cex_or_bridge']]['address'].unique()
print(len(array_address_cex_or_bridge))

32


In [42]:
df_count_seed['cex_or_bridge'] = df_count_seed.index.isin(array_address_cex_or_bridge)
df_count_seed.head(2)

,EOA,cex_or_bridge
from_address,,
0x80c67432656d59144ceff962e8faf8926599bcf8,3148,True
0xacd03d601e5bb1b275bb94076ff46ed9d753435a,2980,True


Here we compute some metric to see how many sybil could be flagged with the new seed clustering algorithm.
These is not the best approach because we should look at these suspicious addresses and their votes to see if they are sybil or not.

In [64]:
df_suspicous_seed = df_count_seed.loc[np.logical_and(~df_count_seed['cex_or_bridge'], df_count_seed['EOA'] > 2)]
print(df_suspicous_seed.head(2))
print(f'number of suspicious seeding wallets: {df_suspicous_seed.shape[0]}')
print(f'number of expected sybils: {df_suspicous_seed["EOA"].sum()}')

                                            EOA  cex_or_bridge
from_address                                                  
0x17f52a50227ea7aa6042eaae44c84a9a9d59d092   67          False
0x4a1939dc1de524ff01980a911f0f4d65ce7a27ba   25          False
number of suspicious seeding wallets: 123
number of expected sybils: 636


In [76]:
df_suspicous_seed = df_count_seed.loc[np.logical_and(~df_count_seed['cex_or_bridge'], df_count_seed['EOA'] > 1)]
print(f'number of suspicious seeding wallets: {df_suspicous_seed.shape[0]}')
print(f'number of expected sybils: {df_suspicous_seed["EOA"].sum()}')

number of suspicious seeding wallets: 313
number of expected sybils: 1016


Above 3 seeding of voters we can expect the voter address to be a sybil. At 2 we may not be sure so the best way is to look at the votes.

In [217]:
array_suspicous_seed = df_suspicous_seed.index.values
array_sus_seed_eoa = df_seed[df_seed['EOA'].isin(array_suspicous_seed)].EOA.unique()
array_sus_seed_from = df_seed[df_seed['from_address'].isin(array_suspicous_seed)].from_address.unique()
suspicious_voter = df_seed[np.logical_or(df_seed['EOA'].isin(array_suspicous_seed), df_seed['from_address'].isin(array_suspicous_seed))]['EOA'].unique()
print(f'number of suspicious voters: {len(suspicious_voter)}')
print(f'number of suspicious_seeder {len(array_suspicous_seed)}')

number of suspicious voters: 1200
number of suspicious_seeder 313


In [180]:
# minimum number of wallets seeded
df_seed[df_seed['from_address'].isin(array_suspicous_seed)].groupby('from_address').count()['EOA'].min()

2

In [110]:
df_votes['has_seed_cluster'] = df_votes['voter'].isin(suspicious_voter)

In [122]:
df_suspicious_votes = df_votes.loc[df_votes['has_seed_cluster']].copy()


In [218]:
def assign_cluster(address, df_seed, array_sus_seed_eoa, array_sus_seed_from, array_address_cex_or_bridge):
    cluster_adddress = ''
    seed_address = df_seed.loc[df_seed['EOA'] == address, 'from_address'].values[0]

    if (seed_address in array_address_cex_or_bridge):
        cluster_adddress = address
    else:
        if address in array_sus_seed_eoa:
            cluster_adddress = address
        else:
            if seed_address in array_sus_seed_from:
                cluster_adddress = seed_address
            else:
                cluster_adddress = address

    # if (address in array_suspicous_seed):
    #     if seed_address in array_address_cex_or_bridge:
    #         cluster_adddress = address
    #     elif seed_address in array_suspicous_seed:
    #         cluster_adddress = seed_address
    #     else:
    #         cluster_adddress = seed_address
    # else:
    #     cluster_adddress = seed_address

    return cluster_adddress

In [220]:
# Either the cluster corresponds to the seed wallet or the voter is the seed wallet
# we have to remove the case where the voter seed wallet is an exchange then we assign the voter address
df_suspicious_votes['cluster_address'] = df_suspicious_votes['voter'].apply(lambda x: assign_cluster(x, df_seed, array_sus_seed_eoa, array_sus_seed_from, array_address_cex_or_bridge))

In [185]:
print(df_suspicious_votes['cluster_address'].unique().shape)
print(df_suspicious_votes.shape)

(385,)
(4745, 35)


In [186]:
# verify nobody votes from a cex
df_suspicious_votes['voter'].isin(df_count_seed[df_count_seed['cex_or_bridge']].EOA.values).sum()

0

In [187]:
# verify no cluster address from a cex
df_suspicious_votes['cluster_address'].isin(df_count_seed[df_count_seed['cex_or_bridge']].EOA.values).sum()

0

In [228]:
'0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5' in array_sus_seed_eoa

False

In [227]:
'0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5' in array_sus_seed_from

True

In [212]:
'0xf86aced730d6fa0340cd811f7b88adc13b22f5c2' in array_address_cex_or_bridge

False

In [229]:
df_seed[df_seed['from_address'] == '0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5']

,EOA,from_address,to_address
2695,0x316d05e01f277e7f028c1b1ba1fbcfb5336ec5f7,0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5,0x316d05e01f277e7f028c1b1ba1fbcfb5336ec5f7
6583,0x787075e3b69d7af519fe3abeaa8f83adc4ba7c38,0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5,0x787075e3b69d7af519fe3abeaa8f83adc4ba7c38


In [230]:
df_seed[df_seed['EOA'] == '0x316d05e01f277e7f028c1b1ba1fbcfb5336ec5f7']

,EOA,from_address,to_address
2695,0x316d05e01f277e7f028c1b1ba1fbcfb5336ec5f7,0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5,0x316d05e01f277e7f028c1b1ba1fbcfb5336ec5f7


In [231]:
df_suspicious_votes[df_suspicious_votes['voter'] == '0x316d05e01f277e7f028c1b1ba1fbcfb5336ec5f7']['cluster_address'].values

array(['0x316d05e01f277e7f028c1b1ba1fbcfb5336ec5f7'], dtype=object)

In [232]:
df_suspicious_votes[df_suspicious_votes['voter'] == '0x787075e3b69d7af519fe3abeaa8f83adc4ba7c38']['cluster_address'].values

array(['0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5'], dtype=object)

In [224]:
df_suspicious_votes[df_suspicious_votes['cluster_address'] == '0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5']['voter'].values

array(['0x787075e3b69d7af519fe3abeaa8f83adc4ba7c38'], dtype=object)

In [223]:
df_suspicious_votes.groupby('cluster_address').count().sort_values('tx_hash').head(23)

,block_timestamp,tx_hash,voter,project,amount_usd,__row_index,seed_same_naive,seed_same,seed_suspicious,less_5_tx,...,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,flagged,stakeridoo_detected,doge_detected,odc_detected,has_seed_cluster
cluster_address,,,,,,,,,,,,,,,,,,,,,
0x823f005cecbb6f6ffd8f0d72d7ba9084064396f5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0xf855f3733fc479e549914068491a4d0cf0e57729,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0x36808a5bee04449cd040464836fbec26ab36004b,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0xc485a40c286182e4767d1cf382c382011dd82bf6,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
0x124ac4be876bb5826cb3c9600b13611ae63eb427,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
0x204c4f3788320960abd329dab51390ab37ddc4eb,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
0x9b32e12d0678628aee378dbdf92f7f66f3f4df80,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
0x14dc78bf7e2021cf7d080b7c0eb8d644e1a8751d,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
0x9f35830b5bd350bbaedcfe84ef5cc151e5ee31eb,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [128]:
df_suspicious_votes['cluster_address'].isna().sum()

0

In [88]:
df_suspicious_votes_seed = df_suspicious_votes.merge(df_seed, left_on='voter', right_on='EOA', how='left').drop(columns=['EOA', 'to_address'])

In [104]:
df_suspicious_votes_seed

,block_timestamp,tx_hash,voter,project,amount_usd,__row_index,seed_same_naive,seed_same,seed_suspicious,less_5_tx,...,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,flagged,stakeridoo_detected,doge_detected,odc_detected,from_address
0,2023-06-27T23:08:53.000Z,0x7e66bced0b92d299c192fc8316c8c869ca5032dd8dd3...,0x27bd053afa71c286c024be7b468921c8672724d8,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.503751,24,True,True,False,False,...,0,0,False,False,False,False,False,False,False,0x2c32c167ac5205618d831e83517571955f31ead8
1,2023-06-27T23:09:59.000Z,0x636b976a55ce21ca6b94a13e836db83ab76299f86f3a...,0x2c284b3b775dedbbcad12a6f0bc54cb928adc2b1,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.503751,25,True,True,False,False,...,0,0,False,False,False,False,False,False,False,0xf7d36a57cbe154fd3d9134cac44ab49019180e12
2,2023-06-27T23:06:27.000Z,0xf0566c2829378030f543493c7dd1c154db274729d7ff...,0xf7d36a57cbe154fd3d9134cac44ab49019180e12,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.503751,36,True,False,True,False,...,0,0,True,False,False,True,False,False,True,0x34c7d0638dab854c7246623575b7839031b4fef6
3,2023-06-27T20:00:41.000Z,0xd02267b66695a171351d63d8bcc762d1865cd40e9b8f...,0x6fcb576b8eaf472221f3154f6a3eaafed2fa27d5,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,5.639068,117,True,True,False,False,...,0,0,True,False,False,True,False,False,True,0xcd98aa3437f953259e6983ebb8fe47b5f50317b4
4,2023-06-27T06:00:03.000Z,0xf68ecd1443e703cae95c901e26914c7e056fb26d6fe7...,0x0464a9f2413a104717634270c49c4041e988078e,0x91fbd3447077bfd97f5af4d2c033ecbc25788f89,1.165407,136,True,True,False,False,...,0,0,False,False,False,False,False,False,False,0xd679c7d2e1217f5902b7521be89874d85f4f4301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4740,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0xc98786d5a7a03c1e74affcb97ff7ef8a710da09b,1.124823,57342,True,True,False,False,...,0,0,False,False,False,False,False,False,False,0x12cc2273ae9929fde04a8960cb64aa72ef43c078
4741,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0x7904667c340601aab73939372c016dc5102732a2,1.124823,57343,True,True,False,False,...,0,0,False,False,False,False,False,False,False,0x12cc2273ae9929fde04a8960cb64aa72ef43c078
4742,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0x298f7f66ba43f0efecf4bc324b0016f822c783a4,1.124823,57344,True,True,False,False,...,0,0,False,False,False,False,False,False,False,0x12cc2273ae9929fde04a8960cb64aa72ef43c078
4743,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0xdc71a1bdeabd3c347dc21e9354aff91ad375eb97,1.124823,57345,True,True,False,False,...,0,0,False,False,False,False,False,False,False,0x12cc2273ae9929fde04a8960cb64aa72ef43c078


In [103]:
'0x80c67432656d59144ceff962e8faf8926599bcf8' in suspicious_voter

False

In [99]:
df_suspicious_votes_seed.groupby('from_address').apply(lambda x: len(x['project'].unique())).sort_values(ascending=False)

from_address
0x80c67432656d59144ceff962e8faf8926599bcf8    33
0xe6d20b8d3e85df7b57a15fd42b86df1c68e6d967    33
0x17f52a50227ea7aa6042eaae44c84a9a9d59d092    33
0x95e301c5aa86f1c9c5c8a04fccff916de7cb6ea3    33
0xd317fc3fbead8e95f5b75145f9dc5e1c7b815856    33
                                              ..
0xe75a587a24251d722e95e2bc5f2c84dfda5da405     1
0x7029fd56efe10959dd0efd22e1db09fbc4e0174b     1
0xe906c99fb03bbddd92652ee9a0e5655dcc389d98     1
0x44444444145b711fbbdbc2f7bc82952d587b2f17     1
0xff97ec2ebe34655acb18e85b38a4248677fa3efc     1
Length: 394, dtype: int64

In [86]:
df_suspicious_votes

,block_timestamp,tx_hash,voter,project,amount_usd,__row_index,seed_same_naive,seed_same,seed_suspicious,less_5_tx,...,is_airdrop_master,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,flagged,stakeridoo_detected,doge_detected,odc_detected
24,2023-06-27T23:08:53.000Z,0x7e66bced0b92d299c192fc8316c8c869ca5032dd8dd3...,0x27bd053afa71c286c024be7b468921c8672724d8,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.503751,24,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
25,2023-06-27T23:09:59.000Z,0x636b976a55ce21ca6b94a13e836db83ab76299f86f3a...,0x2c284b3b775dedbbcad12a6f0bc54cb928adc2b1,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.503751,25,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
36,2023-06-27T23:06:27.000Z,0xf0566c2829378030f543493c7dd1c154db274729d7ff...,0xf7d36a57cbe154fd3d9134cac44ab49019180e12,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.503751,36,True,False,True,False,...,False,0,0,True,False,False,True,False,False,True
117,2023-06-27T20:00:41.000Z,0xd02267b66695a171351d63d8bcc762d1865cd40e9b8f...,0x6fcb576b8eaf472221f3154f6a3eaafed2fa27d5,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,5.639068,117,True,True,False,False,...,False,0,0,True,False,False,True,False,False,True
136,2023-06-27T06:00:03.000Z,0xf68ecd1443e703cae95c901e26914c7e056fb26d6fe7...,0x0464a9f2413a104717634270c49c4041e988078e,0x91fbd3447077bfd97f5af4d2c033ecbc25788f89,1.165407,136,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57342,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0xc98786d5a7a03c1e74affcb97ff7ef8a710da09b,1.124823,57342,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
57343,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0x7904667c340601aab73939372c016dc5102732a2,1.124823,57343,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
57344,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0x298f7f66ba43f0efecf4bc324b0016f822c783a4,1.124823,57344,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
57345,2023-06-26T15:01:09.000Z,0xb7c25044841e7466502c98fbc44c66f2e8c72377d9ce...,0xe19586f58770413fa0fcf6b441408d0a458a2ea5,0xdc71a1bdeabd3c347dc21e9354aff91ad375eb97,1.124823,57345,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False


In [80]:
df_seed

,EOA,from_address,to_address
0,0x000000006f457c0f8f560333d9c2877287d92a92,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x000000006f457c0f8f560333d9c2877287d92a92
1,0x000128fa45d79dc9af8016da242781f12c363fd5,0xe4edb277e41dc89ab076a1f049f4a3efa700bce8,0x000128fa45d79dc9af8016da242781f12c363fd5
2,0x000569d03483abb8949f729908465e65c4bdb243,0x80c67432656d59144ceff962e8faf8926599bcf8,0x000569d03483abb8949f729908465e65c4bdb243
3,0x0015f1d02b55ec9edd7536ca628fd30361252ebf,0x2d2cc0eb095e43204e0c087e07dbf95909650939,0x0015f1d02b55ec9edd7536ca628fd30361252ebf
4,0x0015f91e93996d7e5979bf32391a8be59aadd606,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x0015f91e93996d7e5979bf32391a8be59aadd606
...,...,...,...
13815,0xffec9e15a95845b6720a7e7ea5f65e97eaead3e8,0x80c67432656d59144ceff962e8faf8926599bcf8,0xffec9e15a95845b6720a7e7ea5f65e97eaead3e8
13816,0xfff00f4eb5c003db1f59f86428efd6b7cc2ae5f7,0xe76a0b8533c99545d74aa5133bc212067076e597,0xfff00f4eb5c003db1f59f86428efd6b7cc2ae5f7
13817,0xfffa92ec41925b4f4fa44cef294083cbe16b02f5,0x80c67432656d59144ceff962e8faf8926599bcf8,0xfffa92ec41925b4f4fa44cef294083cbe16b02f5
13818,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d,0xdf362413e0125a93c57c31472f5cc02bebb99306,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d


In [59]:
df_suspicious_votes

,block_timestamp,tx_hash,voter,project,amount_usd,__row_index,seed_same_naive,seed_same,seed_suspicious,less_5_tx,...,is_airdrop_master,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,flagged,stakeridoo_detected,doge_detected,odc_detected
36,2023-06-27T23:06:27.000Z,0xf0566c2829378030f543493c7dd1c154db274729d7ff...,0xf7d36a57cbe154fd3d9134cac44ab49019180e12,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.503751,36,True,False,True,False,...,False,0,0,True,False,False,True,False,False,True
136,2023-06-27T06:00:03.000Z,0xf68ecd1443e703cae95c901e26914c7e056fb26d6fe7...,0x0464a9f2413a104717634270c49c4041e988078e,0x91fbd3447077bfd97f5af4d2c033ecbc25788f89,1.165407,136,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
137,2023-06-27T06:00:03.000Z,0xf68ecd1443e703cae95c901e26914c7e056fb26d6fe7...,0x0464a9f2413a104717634270c49c4041e988078e,0x13257e783dc8ded7e227a28edb0428c42d31202e,1.165407,137,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
138,2023-06-27T06:00:03.000Z,0xf68ecd1443e703cae95c901e26914c7e056fb26d6fe7...,0x0464a9f2413a104717634270c49c4041e988078e,0xa30ab83e693ad49f3f651085dad11d049c818923,1.165407,138,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
139,2023-06-27T06:00:03.000Z,0xf68ecd1443e703cae95c901e26914c7e056fb26d6fe7...,0x0464a9f2413a104717634270c49c4041e988078e,0xe1887ff140bfa9d3b45d0b2077b7471124acd242,1.165407,139,True,True,False,False,...,False,0,0,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56812,2023-06-14T01:34:15.000Z,0x6c40d435ca3e578dfb7aad00c3f93aad459d82dce8c7...,0x6bb275953d6297261386a8eac44db37e8f278ebf,0xc98786d5a7a03c1e74affcb97ff7ef8a710da09b,1.727713,56812,True,True,False,False,...,False,0,0,False,False,False,False,True,False,True
56813,2023-06-14T01:34:15.000Z,0x6c40d435ca3e578dfb7aad00c3f93aad459d82dce8c7...,0x6bb275953d6297261386a8eac44db37e8f278ebf,0xc4450c8d1009160883e44f24d66d92436ae4b4c5,1.727713,56813,True,True,False,False,...,False,0,0,False,False,False,False,True,False,True
56814,2023-06-14T01:34:15.000Z,0x6c40d435ca3e578dfb7aad00c3f93aad459d82dce8c7...,0x6bb275953d6297261386a8eac44db37e8f278ebf,0x745ce2af76e9a6eba65cc0cacaa9ea109bb7fabd,1.727713,56814,True,True,False,False,...,False,0,0,False,False,False,False,True,False,True
56835,2023-06-19T05:55:29.000Z,0x280b17632e4799a6ff2a8a2c089dca07185afe8a113b...,0x8adfc6e6e6da6ed9af7f094b3f3115ed72048395,0xeb0cf83c80e4b4cd82196dac94e2c579672b6b1c,5.176952,56835,True,True,False,False,...,True,0,0,False,False,False,True,False,False,True


In [58]:
df_suspicious_votes.groupby('project').count().sort_values(by='voter', ascending=False).head(10)

,block_timestamp,tx_hash,voter,amount_usd,__row_index,seed_same_naive,seed_same,seed_suspicious,less_5_tx,less_10_tx,...,is_airdrop_master,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,flagged,stakeridoo_detected,doge_detected,odc_detected
project,,,,,,,,,,,,,,,,,,,,,
0xc98786d5a7a03c1e74affcb97ff7ef8a710da09b,76,76,76,76,76,76,76,76,76,76,...,76,76,76,76,76,76,76,76,76,76
0x745ce2af76e9a6eba65cc0cacaa9ea109bb7fabd,70,70,70,70,70,70,70,70,70,70,...,70,70,70,70,70,70,70,70,70,70
0xe1887ff140bfa9d3b45d0b2077b7471124acd242,70,70,70,70,70,70,70,70,70,70,...,70,70,70,70,70,70,70,70,70,70
0xeb0cf83c80e4b4cd82196dac94e2c579672b6b1c,65,65,65,65,65,65,65,65,65,65,...,65,65,65,65,65,65,65,65,65,65
0x88c379caef7965c49b17c2a18c43af36f213fbd5,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
0x7904667c340601aab73939372c016dc5102732a2,45,45,45,45,45,45,45,45,45,45,...,45,45,45,45,45,45,45,45,45,45
0x13257e783dc8ded7e227a28edb0428c42d31202e,41,41,41,41,41,41,41,41,41,41,...,41,41,41,41,41,41,41,41,41,41
0xa51e0a99b53d5b00937a2631d5865468a3543b7d,39,39,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
0x298f7f66ba43f0efecf4bc324b0016f822c783a4,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
